##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tutorial do componente TFX Keras

***Uma introdução ao TensorFlow Extended (TFX) componente por componente***

Observação: recomendamos executar este tutorial em um notebook Colab, sem necessidade de configuração! Basta clicar em “Executar no Google Colab”.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/components_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/components_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/components_keras.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a></td>
<td>     <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/components_keras.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

Este tutorial baseado em Colab percorrerá interativamente cada componente integrado do TensorFlow Extended (TFX).

Ele cobre todas as etapas de um pipeline de aprendizado de máquina de ponta a ponta, desde a ingestão de dados até o envio de um modelo até a disponibilização do serviço.

Quando terminar, o conteúdo deste notebook poderá ser exportado automaticamente como código-fonte do pipeline TFX, que você poderá orquestrar com Apache Airflow e Apache Beam.

Observação: este notebook demonstra o uso de modelos Keras nativos em pipelines do TFX. **O TFX oferece suporte apenas à versão TensorFlow 2 do Keras**.

## Contexto

Este notebook demonstra como usar o TFX num ambiente Jupyter/Colab. Aqui, percorremos o exemplo do Chicago Taxi em um notebook interativo.

Trabalhar com um notebook interativo é uma maneira útil de se familiarizar com a estrutura de um pipeline TFX. Também é útil ao desenvolver seus próprios pipelines como um ambiente de desenvolvimento leve, mas você deve estar ciente de que há diferenças na forma como os notebooks interativos são orquestrados e como eles acessam artefatos de metadados.

### Orquestração

Numa implantação TFX em produção, você usará um orquestrador como Apache Airflow, Kubeflow Pipelines ou Apache Beam para orquestrar o grafo de um pipeline predefinido de componentes do TFX. Num notebook interativo, o próprio notebook é o orquestrador, executando cada componente do TFX conforme você executa as células do notebook.

### Metadados

Numa implantação do TFX em produção, você acessará metadados por meio da API ML Metadata (MLMD). O MLMD armazena propriedades de metadados num banco de dados como MySQL ou SQLite e armazena as payloads de metadados em armazenamento persistente, como seu sistema de arquivos. Num notebook interativo, as propriedades e as cargas são armazenadas num banco de dados SQLite efêmero no diretório `/tmp` do notebook Jupyter ou do servidor Colab.

## Configuração

Primeiro, instalamos e importamos os pacotes necessários, configuramos caminhos e baixamos os dados.

### Atualize o Pip

Para evitar a atualização do Pip num sistema ao executar localmente, garanta que estamos executando no Colab. Os sistemas locais podem, claro, ser atualizados separadamente.

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

### Instale o TFX

**Observação: no Google Colab, devido a atualizações de pacotes, na primeira vez que você executar esta célula você deverá reiniciar o runtime (Runtime &gt; Restart runtime...).**

In [ ]:
!pip install tfx

### Desinstale o shapely

TODO(b/263441833) Esta é uma solução temporária para evitar um ImportError. Em última análise, isto deverá ser resolvido com suporte a uma versão mais recente do Bigquery, em vez de desinstalar outras dependências extras.

In [ ]:
!pip uninstall shapely -y

## Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime ("Runtime &gt; Restart runtime ..."). Isso é necessário devido à maneira como o Colab carrega os pacotes.

### Importe os pacotes

Importamos os pacotes necessários, incluindo classes de componentes TFX padrão.

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

Vamos verificar as versões das bibliotecas.

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

### Configure os caminhos do pipeline

In [ ]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/taxi_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

### Baixe dados de exemplo

Baixamos o dataset de exemplo para uso em nosso pipeline TFX.

O dataset que estamos usando é o [dataset Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) disponibilizado pela cidade de Chicago. As colunas neste dataset são:

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

Com esse dataset, construiremos um modelo que irá prever as `tips` (gorjetas) de uma viagem.

In [ ]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

Dê uma olhada rápida no arquivo CSV.

In [ ]:
!head {_data_filepath}

*Observação: Este site fornece aplicativos que utilizam dados que foram modificados para uso a partir de sua fonte original obtida em www.cityofchicago.org, site oficial da cidade de Chicago. A cidade de Chicago não faz nenhuma reivindicação quanto ao conteúdo, precisão, atualidade ou integridade de qualquer um dos dados fornecidos neste site. Os dados fornecidos neste site estão sujeitos a alterações a qualquer momento. Entende-se que os dados fornecidos neste site são utilizados por sua conta e risco.*

### Crie o InteractiveContext

Por último, criamos um InteractiveContext, que nos permitirá executar componentes TFX interativamente neste notebook.

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

## Execute componentes TFX interativamente

Nas células a seguir, criamos os componentes TFX um por um, executamos cada um deles e visualizamos seus artefatos de saída.

### ExampleGen

O componente `ExampleGen` geralmente está no início de um pipeline TFX. Ele vai:

1. Dividir os dados em datasets de treinamento e avaliação (por padrão, 2/3 de treinamento + 1/3 de avaliação)
2. Converter dados para o formato `tf.Example` (saiba mais [aqui](https://www.tensorflow.org/tutorials/load_data/tfrecord))
3. Copiar dados para o diretório `_tfx_root` para que outros componentes possam acessar

O `ExampleGen` toma como entrada o caminho para sua fonte de dados. No nosso caso, este é o caminho `_data_root` que contém o CSV baixado.

Observação: neste notebook, podemos instanciar os componentes um por um e executá-los com `InteractiveContext.run()`. Por outro lado, num ambiente de produção, especificaríamos todos os componentes antecipadamente num `Pipeline` para passar ao orquestrador (veja o [Guia de criação de um pipeline TFX](https://www.tensorflow.org/tfx/guide/build_tfx_pipeline)).

#### Ativando o Cache

Ao usar o `InteractiveContext` num notebook para desenvolver um pipeline, você pode controlar quando componentes individuais armazenarão em cache suas saídas. Configure `enable_cache` como `True` quando desejar reutilizar os artefatos de saída anteriores gerados pelo componente. Defina `enable_cache` como `False` quando desejar recalcular os artefatos de saída de um componente, se estiver fazendo alterações no código, por exemplo.

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

Vamos examinar os artefatos de saída de `ExampleGen`. Este componente produz dois artefatos, exemplos de treinamento e exemplos de avaliação:

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

Também podemos dar uma olhada nos três primeiros exemplos de treinamento:

In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

Agora que `ExampleGen` concluiu a ingestão dos dados, a próxima etapa será a análise dos dados.

### StatisticsGen

O componente `StatisticsGen` computa estatísticas sobre seu dataset para análise de dados, bem como para uso em componentes downstream. Ele usa a biblioteca [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

`StatisticsGen` toma como entrada o dataset que acabamos de ingerir usando `ExampleGen`.

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

Depois que `StatisticsGen` terminar a execução, podemos visualizar as estatísticas geradas. Experimente brincar com os diferentes gráficos!

In [ ]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

O componente `SchemaGen` gera um esquema com base nas suas estatísticas de dados. (Um esquema define os limites, tipos e propriedades esperados das características em seu dataset.) Ele também usa a biblioteca [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

Nota: O esquema gerado automaticamente é o de melhor esforço e tenta apenas inferir propriedades básicas dos dados. Espera-se que você o revise e modifique conforme seja necessário.

O `SchemaGen` terá como entrada as estatísticas que geramos com `StatisticsGen`, observando a divisão de treinamento por padrão.

In [ ]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

Após a execução do `SchemaGen`, podemos visualizar o esquema gerado como uma tabela.

In [ ]:
context.show(schema_gen.outputs['schema'])

Cada característica no seu dataset aparece como uma linha na tabela do esquema, junto com suas propriedades. O esquema também captura todos os valores que uma característica categórica assume, denotados como o seu domínio.

Para saber mais sobre esquemas, consulte a [documentação do SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen).

### ExampleValidator

O componente `ExampleValidator` detecta anomalias em seus dados, com base nas expectativas definidas pelo esquema. Ele também usa a biblioteca [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

O `ExampleValidator` receberá como entrada as estatísticas de `StatisticsGen` e o esquema de `SchemaGen`.

In [ ]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

Após a execução do `ExampleValidator`, podemos visualizar as anomalias como uma tabela.

In [ ]:
context.show(example_validator.outputs['anomalies'])

Na tabela de anomalias, podemos observar que não há anomalias. Isto é o que esperávamos, já que este é o primeiro dataset que analisamos e o esquema foi adaptado a ele. Você deve revisar este esquema – qualquer coisa inesperada significa uma anomalia nos dados. Depois de revisado, o esquema poderá ser usado para proteger dados futuros, e as anomalias produzidas aqui podem ser usadas para depurar o desempenho do modelo, entender como seus dados evoluem ao longo do tempo e identificar erros de dados.

### Transform

O componente `Transform` realiza engenharia de características para treinamento e serviço. Ele usa a biblioteca [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started).

O `Transform` terá como entrada os dados de `ExampleGen`, o esquema de `SchemaGen`, bem como um módulo que contém código Transform definido pelo usuário.

Vejamos abaixo um exemplo de código Transform definido pelo usuário (para uma introdução às APIs TensorFlow Transform, [consulte o tutorial](https://www.tensorflow.org/tfx/tutorials/transform/simple)). Primeiro, definimos algumas constantes para engenharia de características:

Observação: A magia da célula `%%writefile` salvará o conteúdo da célula como um arquivo `.py` no disco. Isso permite que o componente `Transform` carregar seu código como um módulo.


In [ ]:
_taxi_constants_module_file = 'taxi_constants.py'

In [ ]:
%%writefile {_taxi_constants_module_file}

NUMERICAL_FEATURES = ['trip_miles', 'fare', 'trip_seconds']

BUCKET_FEATURES = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]
# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

CATEGORICAL_NUMERICAL_FEATURES = [
    'trip_start_hour', 'trip_start_day', 'trip_start_month',
    'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    'dropoff_community_area'
]

CATEGORICAL_STRING_FEATURES = [
    'payment_type',
    'company',
]

# Number of vocabulary terms used for encoding categorical features.
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized categorical are hashed.
OOV_SIZE = 10

# Keys
LABEL_KEY = 'tips'
FARE_KEY = 'fare'

def t_name(key):
  """
  Rename the feature keys so that they don't clash with the raw keys when
  running the Evaluator component.
  Args:
    key: The original feature key
  Returns:
    key with '_xf' appended
  """
  return key + '_xf'

Em seguida, escrevemos um `preprocessing_fn` que recebe dados brutos como entrada e retorna características transformadas que nosso modelo pode usar para treinar:

In [ ]:
_taxi_transform_module_file = 'taxi_transform.py'

In [ ]:
%%writefile {_taxi_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

_NUMERICAL_FEATURES = taxi_constants.NUMERICAL_FEATURES
_BUCKET_FEATURES = taxi_constants.BUCKET_FEATURES
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_CATEGORICAL_NUMERICAL_FEATURES = taxi_constants.CATEGORICAL_NUMERICAL_FEATURES
_CATEGORICAL_STRING_FEATURES = taxi_constants.CATEGORICAL_STRING_FEATURES
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_FARE_KEY = taxi_constants.FARE_KEY
_LABEL_KEY = taxi_constants.LABEL_KEY


def _make_one_hot(x, key):
  """Make a one-hot tensor to encode categorical features.
  Args:
    X: A dense tensor
    key: A string key for the feature in the input
  Returns:
    A dense one-hot tensor as a float list
  """
  integerized = tft.compute_and_apply_vocabulary(x,
          top_k=_VOCAB_SIZE,
          num_oov_buckets=_OOV_SIZE,
          vocab_filename=key, name=key)
  depth = (
      tft.experimental.get_vocabulary_size_by_name(key) + _OOV_SIZE)
  one_hot_encoded = tf.one_hot(
      integerized,
      depth=tf.cast(depth, tf.int32),
      on_value=1.0,
      off_value=0.0)
  return tf.reshape(one_hot_encoded, [-1, depth])


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[taxi_constants.t_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]), name=key)

  for key in _BUCKET_FEATURES:
    outputs[taxi_constants.t_name(key)] = tf.cast(tft.bucketize(
            _fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT, name=key),
            dtype=tf.float32)

  for key in _CATEGORICAL_STRING_FEATURES:
    outputs[taxi_constants.t_name(key)] = _make_one_hot(_fill_in_missing(inputs[key]), key)

  for key in _CATEGORICAL_NUMERICAL_FEATURES:
    outputs[taxi_constants.t_name(key)] = _make_one_hot(tf.strings.strip(
        tf.strings.as_string(_fill_in_missing(inputs[key]))), key)

  # Was this passenger a big tipper?
  taxi_fare = _fill_in_missing(inputs[_FARE_KEY])
  tips = _fill_in_missing(inputs[_LABEL_KEY])
  outputs[_LABEL_KEY] = tf.where(
      tf.math.is_nan(taxi_fare),
      tf.cast(tf.zeros_like(taxi_fare), tf.int64),
      # Test if the tip was > 20% of the fare.
      tf.cast(
          tf.greater(tips, tf.multiply(taxi_fare, tf.constant(0.2))), tf.int64))

  return outputs

Agora, passamos este código de engenharia de características para o componente `Transform` e o executamos para transformar seus dados.

In [ ]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_taxi_transform_module_file))
context.run(transform, enable_cache=True)

Vamos examinar os artefatos de saída de `Transform`. Este componente produz dois tipos de saídas:

- `transform_graph` é o grafo que pode realizar as operações de pré-processamento (este grafo será incluído nos modelos de serviço e avaliação).
- `transformed_examples` representa os dados pré-processados ​​de treinamento e avaliação.

In [ ]:
transform.outputs

Dê uma olhada no artefato `transform_graph`: ele aponta para um diretório contendo 3 subdiretórios:

In [ ]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

O subdiretório `transformed_metadata` contém o esquema dos dados pré-processados. O subdiretório `transform_fn` contém o grafo de pré-processamento real. O subdiretório `metadata` contém o esquema dos dados originais.

Também podemos dar uma olhada nos três primeiros exemplos transformados:

In [ ]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

Depois que o componente `Transform` transformar seus dados em características, a próxima etapa é treinar um modelo.

### Trainer

O componente `Trainer` treinará um modelo definido por você no TensorFlow. Para usar a API Keras, você precisa especificar o [Generic Trainer](https://github.com/tensorflow/community/blob/master/rfcs/20200117-tfx-generic-trainer.md) por configuração `custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor)` no construtor do Trainer.

O `Trainer` recebe como entrada o esquema do `SchemaGen`, os dados e o grafo transformados de `Transform`, os parâmetros de treinamento, bem como um módulo que contém código de modelo definido pelo usuário.

Vejamos um exemplo de código de modelo definido pelo usuário (para uma introdução às APIs do TensorFlow Estimator, [consulte o tutorial](https://www.tensorflow.org/guide/keras)):

In [ ]:
_taxi_trainer_module_file = 'taxi_trainer.py'

In [ ]:
%%writefile {_taxi_trainer_module_file}

from typing import Dict, List, Text

import os
import glob
from absl import logging

import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput

# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

_LABEL_KEY = taxi_constants.LABEL_KEY

_BATCH_SIZE = 40


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      tf_transform_output.transformed_metadata.schema)

def _get_tf_examples_serving_signature(model, tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    """Returns the output to be used in the serving signature."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time.
    raw_feature_spec.pop(_LABEL_KEY)
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_inference(raw_features)
    logging.info('serve_transformed_features = %s', transformed_features)

    outputs = model(transformed_features)
    # TODO(b/154085620): Convert the predicted labels from the model using a
    # reverse-lookup (opposite of transform.py).
    return {'outputs': outputs}

  return serve_tf_examples_fn


def _get_transform_features_signature(model, tf_transform_output):
  """Returns a serving signature that applies tf.Transform to features."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    """Returns the transformed_features to be fed as input to evaluator."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    logging.info('eval_transformed_features = %s', transformed_features)
    return transformed_features

  return transform_features_fn


def export_serving_model(tf_transform_output, model, output_dir):
  """Exports a keras model for serving.
  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    model: A keras model to export for serving.
    output_dir: A directory where the model will be exported to.
  """
  # The layer has to be saved to the model for keras tracking purpases.
  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output),
      'transform_features':
          _get_transform_features_signature(model, tf_transform_output),
  }

  model.save(output_dir, save_format='tf', signatures=signatures)


def _build_keras_model(tf_transform_output: TFTransformOutput
                       ) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    tf_transform_output: [TFTransformOutput], the outputs from Transform

  Returns:
    A keras Model.
  """
  feature_spec = tf_transform_output.transformed_feature_spec().copy()
  feature_spec.pop(_LABEL_KEY)

  inputs = {}
  for key, spec in feature_spec.items():
    if isinstance(spec, tf.io.VarLenFeature):
      inputs[key] = tf.keras.layers.Input(
          shape=[None], name=key, dtype=spec.dtype, sparse=True)
    elif isinstance(spec, tf.io.FixedLenFeature):
      # TODO(b/208879020): Move into schema such that spec.shape is [1] and not
      # [] for scalars.
      inputs[key] = tf.keras.layers.Input(
          shape=spec.shape or [1], name=key, dtype=spec.dtype)
    else:
      raise ValueError('Spec type is not supported: ', key, spec)
  
  output = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
  output = tf.keras.layers.Dense(100, activation='relu')(output)
  output = tf.keras.layers.Dense(70, activation='relu')(output)
  output = tf.keras.layers.Dense(50, activation='relu')(output)
  output = tf.keras.layers.Dense(20, activation='relu')(output)
  output = tf.keras.layers.Dense(1)(output)
  return tf.keras.Model(inputs=inputs, outputs=output)


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, _BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, _BATCH_SIZE)

  model = _build_keras_model(tf_transform_output)

  model.compile(
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])

  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  # Export the model.
  export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)

Agora, passamos o código deste modelo para o componente `Trainer` e o executamos para treinar o modelo.

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_taxi_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer, enable_cache=True)

#### Analise o treinamento com o TensorBoard

Dê uma olhada no artefato do trainer. Aponta para um diretório que contém os subdiretórios do modelo.

In [ ]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

Opcionalmente, podemos conectar o TensorBoard ao Trainer para analisar as curvas de treinamento do nosso modelo.

In [ ]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

### Evaluator

O componente `Evaluator` calcula métricas de desempenho do modelo no dataset de avaliação. Ele usa a biblioteca [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started). O `Evaluator` também pode validar opcionalmente se um modelo recém-treinado é melhor que o modelo anterior. Isto é útil numa configuração de pipeline de produção onde você pode treinar e validar automaticamente um modelo todos os dias. Neste notebook, treinamos apenas um modelo, então o `Evaluator` irá rotular automaticamente o modelo como "bom".

O `Evaluator` terá como entrada os dados de `ExampleGen`, o modelo treinado de `Trainer` e a configuração de fatiamento. A configuração de fatiamento permite dividir suas métricas em valores de características (por exemplo, como é o desempenho do seu modelo em viagens de táxi que começam às 8h versus 20h?). Veja abaixo um exemplo dessa configuração:

In [ ]:
# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and
        # remove the label_key.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=taxi_constants.LABEL_KEY,
            preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(
            feature_keys=['trip_start_hour'])
    ])

Em seguida, damos esta configuração ao `Evaluator` e o executamos.

In [ ]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver, enable_cache=True)

evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator, enable_cache=True)

Agora vamos examinar os artefatos de saída do `Evaluator`.

In [ ]:
evaluator.outputs

Usando a saída `evaluation` podemos mostrar a visualização padrão das métricas globais em todo o dataset de avaliação.

In [ ]:
context.show(evaluator.outputs['evaluation'])

Para ver a visualização das métricas de avaliação fatiadas, podemos chamar diretamente a biblioteca TensorFlow Model Analysis.

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='trip_start_hour')

Esta visualização mostra as mesmas métricas, mas calculadas em cada valor de característica de `trip_start_hour` em vez de em todo o dataset de avaliação.

O TensorFlow Model Analysis oferece suporte a muitas outras visualizações, como Fairness Indicators (indicadores de imparcialidade) e plotagem de uma série temporal de desempenho do modelo. Para saber mais, veja o [tutorial](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic).

Como adicionamos limites à nossa configuração, a saída de validação também estará disponível. A presença de um artefato `blessing` indica que nosso modelo passou na validação. Como esta é a primeira validação realizada, o candidato é automaticamente abençoado.

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

Agora também podemos verificar o sucesso carregando o registro do resultado da validação:

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

### Pusher

O componente `Pusher` geralmente está no final de um pipeline do TFX. Ele verifica se um modelo passou na validação e, em caso afirmativo, exporta o modelo para `_serving_model_dir`.

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)

Vamos examinar os artefatos de saída do `Pusher`.

In [ ]:
pusher.outputs

Em particular, o Pusher exportará seu modelo no formato SavedModel, que se parece com o seguinte:

In [ ]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

Terminamos nosso tour pelos componentes integrados do TFX!